In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [2]:
import nltk
from nltk.corpus import wordnet

In [3]:
PATH = ["../dataset/train.tsv","../dataset/valid.tsv","../dataset/test.tsv", "../dataset/test_no_labels.tsv"]

In [4]:
train = pd.read_csv(PATH[0], sep="\t")
valid = pd.read_csv(PATH[1], sep="\t")
test = pd.read_csv(PATH[2], sep="\t")

In [5]:
train_data = train.preprocessed_tweet_text_no_link
train_data = train_data.append(valid.preprocessed_tweet_text_no_link)
train_data = train_data.reset_index(drop=True)
train_data.shape

(12810,)

In [6]:
test_data = test.preprocessed_tweet_text_no_link
test_data = test_data.reset_index(drop=True)
test_data.shape

(1775,)

In [7]:
vectorizer = TextVectorization(max_tokens=26000, output_sequence_length = 128, split='whitespace', standardize='lower_and_strip_punctuation', output_mode="int")
ds = tf.data.Dataset.from_tensor_slices(train_data)
vectorizer.adapt(ds.batch(64))

2022-01-13 12:10:09.333646: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
vectorizer2 = TextVectorization(max_tokens=26000, output_sequence_length = 128, split='whitespace', standardize='lower_and_strip_punctuation', output_mode="int")
ds2 = tf.data.Dataset.from_tensor_slices(test_data)
vectorizer2.adapt(ds2.batch(64))

In [9]:
vocab_train = set(vectorizer.get_vocabulary())
vocab_test = set(vectorizer2.get_vocabulary())

In [10]:
print(len(vocab_test), len(vocab_train))

7532 15319


In [11]:
len(vocab_test - vocab_train)

2862

In [12]:
syns = wordnet.synsets("test")
print(syns[0].definition())

trying something to find out about it


In [13]:
unk = vocab_test - vocab_train

In [14]:
definition_data= []
for el in unk:
    syns = wordnet.synsets(el)
    if len(syns) != 0:
        sentence = "definition of " + el + " is " +syns[0].definition()
        definition_data.append(sentence)
len(definition_data)

1083

In [15]:
df_augm_helper = pd.DataFrame(definition_data, columns=["preprocessed_tweet_text_no_link"])

In [16]:
df_augm_helper["q1_label"] = 1

In [17]:
df_augm_helper.to_csv("../dataset/augm_helper.tsv", index=False, sep="\t")